In [ ]:
import pypsa
import numpy as np
import pandas as pd

"""
The network:

A central bus that connects a solar generator, an onshore and an offshore wind generator, a battery, and a load.

"""

# Create the timeseries index
index = pd.date_range("2011-01-01 00:00", "2011-12-31 21:00", freq="3h")

# Cost assumptions (example values, should be refined)
# TODO: get realistic figures here and in SEK or kSEK
cost_per_mw_solar = 1000  # in $/MW
cost_per_mw_wind = 1200  # in $/MW
cost_per_mwh_storage = 500  # in $/MWh

# Initialize the network
network = pypsa.Network()
network.set_snapshots(index)

# Add the buses
network.add("Bus", "Main bus")

# Add load
# TODO: replace with data from behovskartan
load_profile = np.full(len(index), 300)  # 300 MW constant load for each interval
network.add("Load", "Desired load", bus="Main bus", p_set=load_profile)

# Add generation
# TODO: replace with data from atlite
solar_availability = np.random.rand(len(index))
wind_onshore_availability = np.random.rand(len(index))
wind_offshore_availability = np.random.rand(len(index))

network.add("Generator", "Solar park", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_solar, p_max_pu=solar_availability)

network.add("Generator", "Wind farm onshorea", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_wind, p_max_pu=wind_onshore_availability)

network.add("Generator", "Wind farm offshore", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_wind, p_max_pu=wind_offshore_availability)

# Add storage
network.add("StorageUnit", "Battery", bus="Main bus", p_nom_extendable=True,
            capital_cost=cost_per_mwh_storage)

# Define the objective function (minimize cost)
network.optimize(network.snapshots, solver_name='cbc')

# Output results
print("Optimal size of Solar Park: {:.2f} MW".format(network.generators.p_nom_opt["Solar park"]))
print("Optimal size of Wind Farm (onshore): {:.2f} MW".format(network.generators.p_nom_opt["Wind farm onshore"]))
print("Optimal size of Wind Farm (offshore): {:.2f} MW".format(network.generators.p_nom_opt["Wind farm offshore"]))
print("Optimal size of Battery Storage: {:.2f} MWh".format(network.storage_units.p_nom_opt["Battery"]))